# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = r"C:\Users\DELL\Downloads\Septimo2025\RI\RI_C\Unidades\Unidad 6\wikipedia_text_corpus.csv"

# Load the latest version
""" df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
) """
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [4]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [5]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
#model = SentenceTransformer(MODEL_NAME)
model = SentenceTransformer(MODEL_NAME, device="cuda:0")
# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [6]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages[:5000],
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [7]:
print(embeddings.shape, embeddings.dtype)

(5000, 768) float32


In [8]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [9]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
print(f"Índice FAISS creado con {index.ntotal} vectores")
k=10
D, I = index.search(query_vec, k)


for i in range(k):
    doc_idx = I[0][i]  # El índice que encontró FAISS
    dist = D[0][i]     # La distancia del vector
    
    # Extraemos la información del DataFrame usando el índice
    doc_id = chunks_df.iloc[doc_idx]['doc_id']
    chunk_id = chunks_df.iloc[doc_idx]['chunk_id']
    texto = chunks_df.iloc[doc_idx]['text']
    
    print(f"\n{i+1}. Doc {doc_id}, Chunk {chunk_id} (Distancia L2: {dist:.4f})")
    print(f"{texto[:300]}...")

Índice FAISS creado con 5000 vectores

1. Doc 1, Chunk 0 (Distancia L2: 0.2764)
Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly important in the case of a battery electric vehicle. Some automobiles are fitted wi...

2. Doc 1, Chunk 4 (Distancia L2: 0.3368)
otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeable or non-rechargeable battery of any size down to the smallest button cells which gives an indicatio...

3. Doc 1, Chunk 1 (Distancia L2: 0.3556)
ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the adjacent picture, the ammet

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [10]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

#Conexión instancia  Qdrant
client = QdrantClient(":memory:")

collection_name = "wikipedia_chunks"

# Creamos la colección
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=embeddings.shape[1],  # Dimensión de embeddings
        distance=Distance.COSINE   # Cosine porque embeddings están normalizados
    )
)


True

In [12]:
points = []

for (idx, row), emb in zip(chunks_df.iterrows(), embeddings):
    point = PointStruct(
        id=idx,
        vector=emb.tolist(),
        payload={
            "text": row["text"],
            "doc_id": int(row["doc_id"]),
            "chunk_id": int(row["chunk_id"])
        }
    )
    points.append(point)

batch_size = 100
for i in range(0, len(points), batch_size):
    batch = points[i:i + batch_size]
    client.upsert(
        collection_name=collection_name,
        points=batch
    )
    
print(f"Insertados {len(points)} documentos en Qdrant")

Insertados 5000 documentos en Qdrant


In [13]:
def busqueda_qdrant(query_text, k=10):
    """
    Busca los k documentos más similares usando Qdrant
    """
    # Obtener embedding de la query y aplanar
    query_vec = embed_query(query_text).flatten()
    
    # Búsqueda en Qdrant usando query_points
    response = client.query_points(
        collection_name=collection_name,
        query=query_vec.tolist(),
        limit=k
    )
    
    # Los resultados están en response.points
    results = response.points
    
    # Formatear resultados
    output = []
    for i, hit in enumerate(results):
        output.append({
            'rank': i + 1,
            'id': hit.id,
            'score': hit.score,
            'doc_id': hit.payload.get('doc_id'),
            'chunk_id': hit.payload.get('chunk_id'),
            'text': hit.payload.get('text', "")
        })
    
    return output

In [14]:
query = "Battery measuring"
results = busqueda_qdrant(query, k=10)

print(f"\n{'=' * 80}")
print(f"Query: '{query}'")
print(f"Top {len(results)} resultados con Qdrant")
print(f"{'=' * 80}\n")

for r in results:
    print(f"{r['rank']}. Doc {r['doc_id']}, Chunk {r['chunk_id']} "
          f"(Score: {r['score']:.4f})")
    print(f"  {r['text'][:300]}...")
    print()


Query: 'Battery measuring'
Top 10 resultados con Qdrant

1. Doc 1, Chunk 0 (Score: 0.8618)
  Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly important in the case of a battery electric vehicle. Some automobiles are fitted wi...

2. Doc 1, Chunk 4 (Score: 0.8316)
  otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeable or non-rechargeable battery of any size down to the smallest button cells which gives an indicatio...

3. Doc 1, Chunk 1 (Score: 0.8222)
  ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the adjacent picture, the a

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?


Se usó Cosine (Distance.COSINE) porque los embeddings están normalizados es decir todos tienen la misma magnitud, esto hace que la comparación se base únicamente en la dirección de los vectores.

- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?


Filtrar por metadata en Qdrant fue mucho más sencillo que en FAISS dado que tiene funciones integradas que permiten filtrar los resultados de búsqueda. En cambio en FAISS no tiene esa capacidad porque solo se enfoca en los vectores.

- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?

 El tiempo de búsqueda aumenta linealmente con k, si se pide cantidades enormes como 1000 resultados o más ahí sí empieza a tardar más.

## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [22]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility

# Conexión a Milvus 
connections.connect("default", host="localhost", port="19530")
print("Conectado a Milvus correctamente")

collection_name = "wikipedia_chunks"

# Eliminar colección si existe
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

embedding_dim = 768  # Dimensión de e5-base-v2


fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_dim),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="doc_id", dtype=DataType.INT64),
    FieldSchema(name="chunk_id", dtype=DataType.INT64)
]

schema = CollectionSchema(fields=fields, description="Wikipedia chunks")
collection = Collection(name=collection_name, schema=schema)

print(f"Colección '{collection_name}' creada con éxito")

Conectado a Milvus correctamente
Colección 'wikipedia_chunks' creada con éxito


In [23]:
data_rows = []
for idx, emb, txt, d_id, c_id in zip(range(len(chunks_df)), 
                                     embeddings.tolist(), 
                                     chunks_df["text"].tolist(), 
                                     chunks_df["doc_id"].tolist(), 
                                     chunks_df["chunk_id"].tolist()):
    data_rows.append({
        "id": idx,
        "vector": emb,
        "text": txt,
        "doc_id": int(d_id),
        "chunk_id": int(c_id)
    })

BATCH_SIZE = 500

# Insertarmos en Milvus por lotes
for i in range(0, len(data_rows), BATCH_SIZE):
    batch = data_rows[i : i + BATCH_SIZE]
    
    # Milvus espera una lista de listas (una por cada columna/campo)
    # o una lista de diccionarios si el esquema lo permite. 
    # Aquí transformamos el batch de diccionarios al formato de columnas:
    entities = [
        [row["id"] for row in batch],
        [row["vector"] for row in batch],
        [row["text"] for row in batch],
        [row["doc_id"] for row in batch],
        [row["chunk_id"] for row in batch]
    ]

    collection.insert(entities)

collection.flush()
print(f"Insertados {len(data_rows)} documentos en Milvus en lotes de {BATCH_SIZE}")

Insertados 5000 documentos en Milvus en lotes de 500


In [24]:
#Creación indice 
index_params_precise = {
    "metric_type": "COSINE",
    "index_type": "HNSW",
    "params": {
        "M": 32,              
        "efConstruction": 400  
    }
}

collection.create_index(field_name="embedding", index_params=index_params_precise)
collection.load()

In [25]:
#Búsqueda PRECISA
search_params_precise = {
    "metric_type": "COSINE",
    "params": {"ef": 200}  # Búsqueda más exhaustiva
}

#Búsqueda RÁPIDA
search_params_fast = {
    "metric_type": "COSINE",
    "params": {"ef": 50}  # Búsqueda más rápida pero menos precisa
}

In [26]:
# Función de búsqueda
def busqueda_milvus(query_text, k=5, precise=True):
    """Busca los k documentos más similares usando Milvus"""
    query_vec = embed_query(query_text)
    
    search_params = search_params_precise if precise else search_params_fast
    
    results = collection.search(
        data=query_vec.tolist(),
        anns_field="embedding",
        param=search_params,
        limit=k,
        output_fields=["text", "doc_id", "chunk_id"]
    )
    
    output = []
    for i, hit in enumerate(results[0]):
        output.append({
            'rank': i + 1,
            'id': hit.id,
            'distance': hit.distance,
            'doc_id': hit.entity.get('doc_id'),
            'chunk_id': hit.entity.get('chunk_id'),
            'text': hit.entity.get('text')
        })
    
    return output

In [27]:
print("\nCOMPARACIÓN CONFIGURACIONES\n")

query = "Battery measuring"

# Configuración precisa
import time
start = time.time()
results_precise = busqueda_milvus(query, k=5, precise=True)
time_precise = time.time() - start

# Configuración rápida
start = time.time()
results_fast = busqueda_milvus(query, k=5, precise=False)
time_fast = time.time() - start

print(f"Tiempo búsqueda precisa: {time_precise:.4f}s")
print(f"Tiempo búsqueda rápida: {time_fast:.4f}s")
print(f"Speedup: {time_precise/time_fast:.2f}x")

# Overlap de resultados
ids_precise = [r['id'] for r in results_precise]
ids_fast = [r['id'] for r in results_fast]
overlap = len(set(ids_precise) & set(ids_fast))

print(f"\nOverlap de IDs: {overlap}/5 ({overlap/5*100:.0f}%)")

# Mostrar resultados
print("\nRESULTADOS CONFIGURACIÓN PRECISA")
for r in results_precise:
    print(f"{r['rank']}. Doc {r['doc_id']}, Chunk {r['chunk_id']} (Distancia: {r['distance']:.4f})")
    print(f"   {r['text'][:200]}...\n")


COMPARACIÓN CONFIGURACIONES

Tiempo búsqueda precisa: 0.0624s
Tiempo búsqueda rápida: 0.0558s
Speedup: 1.12x

Overlap de IDs: 5/5 (100%)

RESULTADOS CONFIGURACIÓN PRECISA
1. Doc 1, Chunk 0 (Distancia: 0.8618)
   Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It...

2. Doc 1, Chunk 4 (Distancia: 0.8316)
   otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeabl...

3. Doc 1, Chunk 1 (Distancia: 0.8222)
   ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the...

4. Doc 5, Chunk 0 (Distancia: 0.8174)
   Capacity loss Capacity loss or capacity fading is

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?

Se ajustaron dos parámetros:

El primero corresponde a la búsqueda precisa donde se aumentaron los valores de M y efConstruction, lo que hace que la búsqueda sea más lenta pero más exacta.  

El segundo corresponde a la búsqueda rápida donde se redujo el parámetro ef = 50 durante la búsqueda, lo que permite una ejecución más veloz, aunque con menor precisión.

- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?

Al comparar los resultados de la configuración precisa vs la rápida si el overlap es 5/5 (100%) significa que ambas configuraciones encontraron exactamente los mismos documentos. Pero si es 4/5 (80%) o menos nos indica que la versión rápida se perdió algún resultado.

## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [30]:
import weaviate
from weaviate.classes.config import Property, DataType, Configure

# Conectar a Weaviate local 
client = weaviate.connect_to_local(
    port=8081,
    skip_init_checks=True
)
print("Conectado a Weaviate correctamente")


# Eliminamos colección si existe
try:
    client.collections.delete("WikipediaChunk")
except Exception:
    pass

# Creamos la colección
collection = client.collections.create(
    name="WikipediaChunk",
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="doc_id", data_type=DataType.INT),
        Property(name="chunk_id", data_type=DataType.INT),
    ],
    vectorizer_config=Configure.Vectorizer.none()  
)


print("Colección 'WikipediaChunk' creada correctamente")


Conectado a Weaviate correctamente
Colección 'WikipediaChunk' creada correctamente


In [31]:
# Construimos lista de objetos con datos + embeddings
data_rows = []
for idx, emb, txt, d_id, c_id in zip(
    range(len(chunks_df)),
    embeddings.tolist(),
    chunks_df["text"].tolist(),
    chunks_df["doc_id"].tolist(),
    chunks_df["chunk_id"].tolist()
):
    data_rows.append({
        "properties": {
            "text": txt,
            "doc_id": int(d_id),
            "chunk_id": int(c_id)
        },
        "vector": emb
    })

BATCH_SIZE = 500

# Inserción en lotes 
with collection.batch.fixed_size(BATCH_SIZE) as batch:
    for row in data_rows:
        batch.add_object(
            properties=row["properties"],
            vector=row["vector"]
        )

print(f"Insertados {len(data_rows)} documentos en Weaviate en lotes de {BATCH_SIZE}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_33168\2728931376.py:5: ResourceWarning: unclosed <socket.socket fd=4720, family=23, type=1, proto=0, laddr=('::1', 52875, 0, 0), raddr=('::1', 8081, 0, 0)>
  embeddings.tolist(),


Insertados 5000 documentos en Weaviate en lotes de 500


In [32]:
# Función de búsqueda
def busqueda_weaviate(query_text, k=5):
    """Busca los k documentos más similares usando Weaviate"""
    query_vec = embed_query(query_text)[0]
    
    response = collection.query.near_vector(
        near_vector=query_vec.tolist(),
        limit=k,
        return_metadata=["distance"]
    )
    
    output = []
    for i, obj in enumerate(response.objects):
        output.append({
            'rank': i + 1,
            'uuid': str(obj.uuid),
            'distance': obj.metadata.distance,
            'doc_id': obj.properties['doc_id'],
            'chunk_id': obj.properties['chunk_id'],
            'text': obj.properties['text']
        })
    
    return output

In [37]:
query = "Battery measuring"
results = busqueda_weaviate(query, k=5)

print("="*80)
print(f"Query: '{query}'")
print(f"Top 5 documentos con Weaviate:\n")
print("="*80)
for r in results:
    print(f"\n{r['rank']}. Doc {r['doc_id']}, Chunk {r['chunk_id']}")
    print(f"   UUID: {r['uuid'][:20]}...")
    print(f"   Distance: {r['distance']:.4f}")
    print(f"   {r['text'][:200]}...")


Query: 'Battery measuring'
Top 5 documentos con Weaviate:


1. Doc 1, Chunk 0
   UUID: 29f6c33e-a44a-4107-a...
   Distance: 0.1382
   Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It...

2. Doc 1, Chunk 4
   UUID: e4a8245d-7766-4259-8...
   Distance: 0.1684
   otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeabl...

3. Doc 1, Chunk 1
   UUID: e32420f4-e05d-4d0b-8...
   Distance: 0.1778
   ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the...

4. Doc 5, Chunk 0
   UUID: 0b44a20f-6cc4-49d3-9...
   Distance: 0.1826
   Capacity loss Capacity loss o

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?

La diferencia principal está en la forma en que se manejan los datos es decir en una base de datos se usan tablas con filas y columnas  donde todos los datos tienen la misma estructura mientras que en Weaviate se define un schema donde cada objeto es una entidad completa con sus datos.

- ¿Cómo describirías el trade-off de complejidad vs expresividad?

Se refiere a lo que se gana y lo que se pierde es decir se pierde simplicidad porque el sistema es más complejo y requiere aprender nuevos conceptos, pero por otro lado permite realizar búsquedas mucho más completas y útiles.

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [38]:
import chromadb

#client = chromadb.Client()
client = chromadb.PersistentClient(path="./chroma_db")

collection_name = "wikipedia_chunks"

# Si la colección ya existe la obtenemos si no la creamos
if collection_name in [c.name for c in client.list_collections()]:
    print(f"La colección '{collection_name}' ya existe, usando la existente")
    collection = client.get_collection(collection_name)
else:
    collection = client.create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}
    )
    print(f"Colección '{collection_name}' creada en ChromaDB")

Colección 'wikipedia_chunks' creada en ChromaDB


In [41]:
ids = [str(i) for i in range(len(passages[:5000]))]

documents = chunks_df["text"].tolist()

# Embeddings 
embeddings_list = embeddings.tolist()

metadatas = [
    {
        "doc_id": int(chunks_df.iloc[i]["doc_id"]),
        "chunk_id": int(chunks_df.iloc[i]["chunk_id"])
    }
    for i in range(len(chunks_df))
]

print(f"Datos preparados: {len(ids)} documentos\n")


Datos preparados: 5000 documentos



## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [55]:
import psycopg2
from pgvector.psycopg2 import register_vector

# Conexión a PostgreSQL 
conn = psycopg2.connect(
    dbname="vectordb",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5434"  
)

cur = conn.cursor()

# Creación extensión pgvector 
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

# Registo el tipo vector en psycopg2
register_vector(conn)

# Eliminar tabla si existe
cur.execute("DROP TABLE IF EXISTS documents;")

# Crear tabla documents
cur.execute("""
    CREATE TABLE documents (
        id SERIAL PRIMARY KEY,
        text TEXT,
        embedding vector(768),
        doc_id INTEGER,
        chunk_id INTEGER
    );
""")

conn.commit()

print("Tabla 'documents' creada correctamente")


Tabla 'documents' creada correctamente


In [56]:
# Nos asegúramos que la transacción esté limpia
conn.rollback()
cur = conn.cursor()

print(f"Insertando {len(embeddings)} registros ")


for i, (texto, vector) in enumerate(zip(passages[:5000], embeddings)):
    try:
        cur.execute(
            """
            INSERT INTO documents (text, embedding, doc_id, chunk_id)
            VALUES (%s, %s, %s, %s);
            """,
            (texto, vector.tolist(), i, i) 
        )
    except Exception as e:
        print(f"Error en el registro {i}: {e}")
        conn.rollback()
        break

conn.commit()
print("Inserción completada ")

Insertando 5000 registros 
Inserción completada 


In [57]:
# Crear índice para acelerar búsquedas
cur.execute("""
    CREATE INDEX ON documents
    USING ivfflat (embedding vector_cosine_ops)
    WITH (lists = 100)
""")
conn.commit()


print("Índice ivfflat creado")
# Cerrar cursor inicial
cur.close()

Índice ivfflat creado


In [58]:
def pgvector_search(query_text, k=5):
    """Busca los k documentos más similares usando pgvector"""
    query_vec = embed_query(query_text)[0]  # lista de floats
    
    # Creamos nuevo cursor para esta operación
    cur = conn.cursor()
    
    try:
        cur.execute(
            """
            SELECT
                id,
                text,
                doc_id,
                chunk_id,
                1 - (embedding <=> %s) as cosine_similarity,
                embedding <=> %s as cosine_distance
            FROM documents
            ORDER BY embedding <=> %s
            LIMIT %s
            """,
            (query_vec, query_vec, query_vec, k)
        )
        
        results = cur.fetchall()
        
        output = []
        for i, row in enumerate(results):
            output.append({
                'rank': i + 1,
                'id': row[0],
                'text': row[1],
                'doc_id': row[2],
                'chunk_id': row[3],
                'similarity': row[4],
                'distance': row[5]
            })
        
        return output
    
    finally:
        cur.close()

In [59]:
query = "Battery measuring"
results = pgvector_search(query, k=5)

print("="*80)
print(f"Query: {query}")
print(f"Top 5 documentos con pgvector:\n")
print("="*80)
for r in results:
    print(f"\n{r['rank']}. Doc {r['doc_id']}, Chunk {r['chunk_id']}")
    print(f"   Similarity: {r['similarity']:.4f} | Distance: {r['distance']:.4f}")
    print(f"   {r['text'][:200]}...")

# Cerrar conexión
conn.close()
print("\n" + "="*80)
print("Conexión cerrada correctamente")

Query: Battery measuring
Top 5 documentos con pgvector:


1. Doc 1, Chunk 1
   Similarity: 0.8618 | Distance: 0.1382
   passage: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of c...

2. Doc 5, Chunk 5
   Similarity: 0.8316 | Distance: 0.1684
   passage: otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a re...

3. Doc 2, Chunk 2
   Similarity: 0.8222 | Distance: 0.1778
   passage: ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharge...

4. Doc 1754, Chunk 1754
   Similarity: 0.8145 | Distance: 0.1855
   passage: ciple. A single clamp is used for single-phase measurements; wit

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?


Esta aproximación me parece más explicable porque usa SQL y es fácil de entender dado que  puedo ver claramente qué hace la consulta no hay cosas ocultas, en cambio con las otras pasan más procesos internamente que no se ven tan claros.

- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?


Ofrece muchas ventajas permite combinar la búsqueda por similitud se puede usar WHERE para filtrar datos específicos, JOIN para unir información de varias tablas esto hace que las búsquedas sean más completas.

- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?

Cuando hay muchísimos vectores PostgreSQL no es tan rápido como las bases vectoriales dedicadas y puede volverse más lento si se insertan y actualizan muchos vectores al mismo tiempo.